<a href="https://colab.research.google.com/github/knc6/jarvis-tools-notebooks/blob/master/jarvis-tools-notebooks/kgcnn_jarvis_leaderboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This notebook shows how to add benchmarks to [JARVIS-Leaderboard](https://pages.nist.gov/jarvis_leaderboard/) using [kgcnn](https://github.com/aimat-lab/gcnn_keras) for [exfoliation_energy](https://pages.nist.gov/jarvis_leaderboard/AI/SinglePropertyPrediction/dft_3d_exfoliation_energy/) task.

First, let's install jarvis_leaderboard

In [1]:
pip install  jarvis_leaderboard

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.0/64.0 MB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.8/973.8 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.0/515.0 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.9/239.9 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 4.0 MB/s eta 0:00:00
  Attempting uninstall: markdown
    Found existing installation: Markdown 3.4.3
    Uninstalling Markdown-3.4.3:
      Successfully uninstalled Markdown-3.4.3
  Attempting uninstal

Install kgcnn

In [2]:
import os
if not os.path.exists('gcnn_keras'):
  !git clone https://github.com/aimat-lab/gcnn_keras.git

Cloning into 'gcnn_keras'...
remote: Enumerating objects: 22874, done.
remote: Counting objects: 100% (1854/1854), done.
remote: Compressing objects: 100% (739/739), done.
remote: Total 22874 (delta 1138), reused 1792 (delta 1098), pack-reused 21020
Receiving objects: 100% (22874/22874), 86.04 MiB | 21.78 MiB/s, done.
Resolving deltas: 100% (16258/16258), done.


In [3]:
!pip install -e ./gcnn_keras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/gcnn_keras
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 43.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 42.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 50.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 591.0/591.0 kB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 81.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 357.2/357.2 kB 35.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 69.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

You might have to restart runtime here in case you can't import kgcnn.
 Got to the top menu>>Runtime>>Restart menu

In [1]:
import jarvis_leaderboard

In [2]:
import kgcnn

Let's populate the dataset. Files will be saved in the output_path i.e. 'Out' directory

In [3]:
!jarvis_populate_data.py --benchmark_file AI-SinglePropertyPrediction-exfoliation_energy-dft_3d-test-mae --output_path=Out

benchmark_file AI-SinglePropertyPrediction-exfoliation_energy-dft_3d-test-mae
dataset dft_3d
output_path Out
property exfoliation_energy
method AI
task SinglePropertyPrediction
id_tag jid
dataset file to be used /usr/local/lib/python3.9/dist-packages/jarvis_leaderboard/dataset/AI/SinglePropertyPrediction/dft_3d_exfoliation_energy.json.zip
Obtaining 3D dataset 76k ...
Reference:https://www.nature.com/articles/s41524-020-00440-1
Other versions:https://doi.org/10.6084/m9.figshare.6815699
100% 40.8M/40.8M [00:03<00:00, 12.3MiB/s]
Loading the zipfile...
Loading completed.
number of training samples 650
number of validation samples 81
number of test samples 81


In [4]:
!rm -r exfoliation_en*

rm: cannot remove 'exfoliation_en*': No such file or directory


In [5]:
ls Out/dataset_info.json

Out/dataset_info.json


In [6]:
from jarvis.db.jsonutils import loadjson
dataset_info = loadjson('Out/dataset_info.json')

In [7]:
dataset_info

{'benchmark_file': 'AI-SinglePropertyPrediction-exfoliation_energy-dft_3d-test-mae',
 'dataset': 'dft_3d',
 'output_path': 'Out',
 'prop': 'exfoliation_energy',
 'methods': 'AI',
 'id_tag': 'jid',
 'n_train': 650,
 'n_val': 81,
 'n_test': 81}

In [8]:
n_train = dataset_info['n_train']
n_val = dataset_info['n_val']
n_test = dataset_info['n_test']

In [9]:
from kgcnn.literature.Schnet import make_crystal_model
from tensorflow.keras.optimizers import Adam
null=None
true=True
false=False

ragged=True
model_config = {
    "name": "Schnet",
    "inputs": [
        {'shape': (None,), 'name': "node_number", 'dtype': 'float32', 'ragged': ragged},
        {'shape': (None, 3), 'name': "node_coordinates", 'dtype': 'float32', 'ragged': ragged},
        {'shape': (None, 2), 'name': "range_indices", 'dtype': 'int64', 'ragged': ragged},
        {'shape': (None, 3), 'name': "range_image", 'dtype': 'int64', 'ragged': ragged},
        {'shape': (3, 3), 'name': "graph_lattice", 'dtype': 'float32', 'ragged': False}
    ],
    "input_embedding": {
        "node": {"input_dim": 95, "output_dim": 64}
    },
    "interaction_args": {
        "units": 128, "use_bias": True, "activation": "kgcnn>shifted_softplus", "cfconv_pool": "sum"
    },
    "node_pooling_args": {"pooling_method": "mean"},
    "depth": 4,
    "gauss_args": {"bins": 25, "distance": 5, "offset": 0.0, "sigma": 0.4}, "verbose": 10,
    "last_mlp": {"use_bias": [True, True, True], "units": [128, 64, 1],
                 "activation": ['kgcnn>shifted_softplus', 'kgcnn>shifted_softplus', 'linear']},
    "output_embedding": "graph",
    "use_output_mlp": False,
    "output_mlp": None,  # Last MLP sets output dimension if None.
}

#from: https://github.com/aimat-lab/gcnn_keras/blob/601d5eed029c1ef2c0c60d91bdd910a798ec03fd/training/results/MatProjectJdft2dDataset/Schnet_make_crystal_model/Schnet_hyper.json
x = {"model": {"module_name": "kgcnn.literature.Schnet", "class_name": "make_crystal_model", "config": {"name": "Schnet", "inputs": [{"shape": [null], "name": "node_number", "dtype": "float32", "ragged": true}, {"shape": [null, 3], "name": "node_coordinates", "dtype": "float32", "ragged": true}, {"shape": [null, 2], "name": "range_indices", "dtype": "int64", "ragged": true}, {"shape": [null, 3], "name": "range_image", "dtype": "int64", "ragged": true}, {"shape": [3, 3], "name": "graph_lattice", "dtype": "float32", "ragged": false}], "input_embedding": {"node": {"input_dim": 95, "output_dim": 64}}, "interaction_args": {"units": 128, "use_bias": true, "activation": "kgcnn>shifted_softplus", "cfconv_pool": "sum"}, "node_pooling_args": {"pooling_method": "mean"}, "depth": 4, "gauss_args": {"bins": 25, "distance": 5, "offset": 0.0, "sigma": 0.4}, "verbose": 10, "last_mlp": {"use_bias": [true, true, true], "units": [128, 64, 1], "activation": ["kgcnn>shifted_softplus", "kgcnn>shifted_softplus", "linear"]}, "output_embedding": "graph", "use_output_mlp": false, "output_mlp": null}}, "training": {"cross_validation": {"class_name": "KFold", "config": {"n_splits": 5, "random_state": 42, "shuffle": true}}, "fit": {"batch_size": 32, "epochs": 800, "validation_freq": 10, "verbose": 2, "callbacks": [{"class_name": "kgcnn>LinearLearningRateScheduler", "config": {"learning_rate_start": 0.0005, "learning_rate_stop": 1e-05, "epo_min": 100, "epo": 800, "verbose": 0}}]}, "compile": {"optimizer": {"class_name": "Adam", "config": {"lr": 0.0005}}, "loss": "mean_absolute_error"}, "scaler": {"class_name": "StandardScaler", "module_name": "kgcnn.data.transform.scaler.scaler", "config": {"with_std": true, "with_mean": true, "copy": true}}, "multi_target_indices": null}, "data": {"dataset": {"class_name": "MatProjectJdft2dDataset", "module_name": "kgcnn.data.datasets.MatProjectJdft2dDataset", "config": {}, "methods": [{"map_list": {"method": "set_range_periodic", "max_distance": 5}}]}, "data_unit": "meV/atom"}, "info": {"postfix": "", "postfix_file": "", "kgcnn_version": "2.2.3"}}
model_config = x['model']['config']


model = make_crystal_model(**model_config)
model.compile(
    loss="mean_absolute_error",
    optimizer=Adam(learning_rate=1e-04),
    metrics=["mean_absolute_error"],
)

INFO:kgcnn.model.utils:Updated model kwargs:
INFO:kgcnn.model.utils:{'name': 'Schnet', 'inputs': [{'shape': [None], 'name': 'node_number', 'dtype': 'float32', 'ragged': True}, {'shape': [None, 3], 'name': 'node_coordinates', 'dtype': 'float32', 'ragged': True}, {'shape': [None, 2], 'name': 'range_indices', 'dtype': 'int64', 'ragged': True}, {'shape': [None, 3], 'name': 'range_image', 'dtype': 'int64', 'ragged': True}, {'shape': [3, 3], 'name': 'graph_lattice', 'dtype': 'float32', 'ragged': False}], 'input_embedding': {'node': {'input_dim': 95, 'output_dim': 64}}, 'make_distance': True, 'expand_distance': True, 'interaction_args': {'units': 128, 'use_bias': True, 'activation': 'kgcnn>shifted_softplus', 'cfconv_pool': 'sum'}, 'node_pooling_args': {'pooling_method': 'mean'}, 'depth': 4, 'gauss_args': {'bins': 25, 'distance': 5, 'offset': 0.0, 'sigma': 0.4}, 'verbose': 10, 'last_mlp': {'use_bias': [True, True, True], 'units': [128, 64, 1], 'activation': ['kgcnn>shifted_softplus', 'kgcnn>sh

We choose one of the SinglePropertyPrediction tasks from the [leaderboard](https://pages.nist.gov/jarvis_leaderboard/AI/SinglePropertyPrediction/). See other examples in the [alignn_model](https://github.com/usnistgov/jarvis_leaderboard/tree/main/jarvis_leaderboard/benchmarks/alignn_model) benchmark folder

In [10]:
import pandas as pd
import os
from jarvis.core.atoms import Atoms
from kgcnn.data.crystal import CrystalDataset
import numpy as np

def prepare_data(dirname='exfoliation_en',populated_data_path='Out'):

    id_prop_path=os.path.join(populated_data_path,'id_prop.csv')
    df=pd.read_csv(id_prop_path,header=None)
    df.columns=['id','target']

   
    #train
    train_dir = dirname+'_train'
    if not os.path.exists(train_dir):
        os.makedirs(train_dir)
    train_cif = os.path.join(train_dir,'CifFiles')
    if not os.path.exists(train_cif):
        os.makedirs(train_cif)
    csv_file = train_dir+'/data.csv'
    print ('train_dir',train_dir)
    f=open(csv_file, "w")
    f.write("file_name,index,label\n")
    targets = []
    for i,ii in df.iterrows():
        pos_path=os.path.join(populated_data_path,ii['id'])
        atoms=Atoms.from_poscar(pos_path)
        pmg = (atoms.pymatgen_converter())
        targets.append(ii['target'])
        fname="file_"+str(ii['id'])+".cif"
        #fname="file_"+str(i)+".cif"
        cif_name = os.path.join(train_cif,fname)
        pmg.to(filename=cif_name, fmt="cif")
        line=fname+','+str(i)+','+str(ii['target'])+'\n'
        f.write(line)
    f.close()
    dataset = CrystalDataset(
        data_directory=train_dir, 
        dataset_name=train_dir, 
        file_name="data.csv", 
        file_directory="CifFiles"
    )
    dataset.prepare_data(file_column_name="file_name", overwrite=True)
    dataset.read_in_memory(label_column_name="label")
    labels = np.expand_dims(dataset.get("graph_labels"), axis=-1)
    dataset.map_list(method="set_range_periodic", max_distance=5.0, max_neighbours=20)
  
    return dataset,labels,df

dataset,labels,df = prepare_data(populated_data_path='Out')



train_dir exfoliation_en_train


INFO:kgcnn.data.exfoliation_en_train:Searching for structure files in 'exfoliation_en_train/CifFiles'
INFO:kgcnn.data.exfoliation_en_train:Read 812 single files.
INFO:kgcnn.data.exfoliation_en_train:... Read .cif file 0 from 812
/usr/local/lib/python3.9/dist-packages/pymatgen/io/cif.py:1147: UserWarning: Issues encountered while parsing CIF: Some fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))
INFO:kgcnn.data.exfoliation_en_train:Exporting as dict for pymatgen ...
INFO:kgcnn.data.exfoliation_en_train:Saving structures as .json ...
INFO:kgcnn.data.exfoliation_en_train:Making node features from structure...
INFO:kgcnn.data.exfoliation_en_train:Reading structures from .json ...
INFO:kgcnn.data.exfoliation_en_train: ... read structures 0 from 812


In [11]:
len(dataset),len(labels)

(812, 812)

In [12]:
train_index=np.arange(0,n_train)
val_index=np.arange(n_train,n_train+n_val)
test_index=np.arange(n_train+n_val,n_train+n_val+n_test)

In [13]:
x_train,y_train=dataset[train_index], labels[train_index]
x_val, y_val = dataset[val_index].tensor(model_config["inputs"]), labels[val_index]
x_test, y_test = dataset[test_index].tensor(model_config["inputs"]), labels[test_index]

In [14]:
model.fit(
    x_train.tensor(model_config["inputs"]),y_train,
    shuffle=False,
    batch_size=10,
    epochs=20,
    verbose=2,
)

Epoch 1/20
65/65 - 10s - loss: 96.8120 - mean_absolute_error: 96.8120 - 10s/epoch - 158ms/step
Epoch 2/20
65/65 - 6s - loss: 61.8695 - mean_absolute_error: 61.8695 - 6s/epoch - 93ms/step
Epoch 3/20
65/65 - 3s - loss: 58.5133 - mean_absolute_error: 58.5133 - 3s/epoch - 49ms/step
Epoch 4/20
65/65 - 3s - loss: 56.1235 - mean_absolute_error: 56.1235 - 3s/epoch - 49ms/step
Epoch 5/20
65/65 - 4s - loss: 54.3929 - mean_absolute_error: 54.3929 - 4s/epoch - 67ms/step
Epoch 6/20
65/65 - 5s - loss: 52.5338 - mean_absolute_error: 52.5338 - 5s/epoch - 76ms/step
Epoch 7/20
65/65 - 3s - loss: 51.4932 - mean_absolute_error: 51.4932 - 3s/epoch - 49ms/step
Epoch 8/20
65/65 - 6s - loss: 50.0960 - mean_absolute_error: 50.0960 - 6s/epoch - 94ms/step
Epoch 9/20
65/65 - 6s - loss: 49.5828 - mean_absolute_error: 49.5828 - 6s/epoch - 92ms/step
Epoch 10/20
65/65 - 3s - loss: 48.5559 - mean_absolute_error: 48.5559 - 3s/epoch - 49ms/step
Epoch 11/20
65/65 - 5s - loss: 47.8889 - mean_absolute_error: 47.8889 - 5s/e

In [15]:
val_pred = model.predict(x_val)
test_pred = model.predict(x_test)

3/3 [==============================] - 0s 109ms/step


In [16]:
from sklearn.metrics import mean_absolute_error
print (mean_absolute_error(y_val,val_pred),mean_absolute_error(y_test,test_pred))

42.8312237360448 40.64849182317286


In [17]:
len(test_pred)

81

Prepare the csv file with six components in the filename.

In [18]:
df_test = df[-n_test:]
f=open('AI-SinglePropertyPrediction-exfoliation_energy-dft_3d-test-mae.csv','w')
f.write('id,prediction\n')
for i in range(len(df_test)):
    #print (i)
    jid = df_test.iloc[i]['id']
    target = df_test.iloc[i]['target']
    #print(jid,target,y_test[i][0],test_pred[i][0])
    line=jid+','+str(test_pred[i][0])+'\n'
    f.write(line)
f.close()

In [19]:
x = pd.read_csv('AI-SinglePropertyPrediction-exfoliation_energy-dft_3d-test-mae.csv')

In [20]:
x

,id,prediction
0,JVASP-12918,128.28584
1,JVASP-2035,82.68650
2,JVASP-13942,76.92734
3,JVASP-278,104.32003
4,JVASP-10173,111.85178
...,...,...
76,JVASP-4364,68.76621
77,JVASP-29480,69.19061
78,JVASP-28375,66.89698
79,JVASP-590,70.43225


In [21]:
import os
cwd=os.getcwd()
os.makedirs('kgcnn_schnet2')
os.chdir('kgcnn_schnet2')

In [22]:
!cp ../AI-SinglePropertyPrediction-exfoliation_energy-dft_3d-test-mae.csv .

In [23]:
!wget https://raw.githubusercontent.com/usnistgov/jarvis_leaderboard/main/jarvis_leaderboard/benchmarks/alignn_model/metadata.json

--2023-04-26 04:59:59--  https://raw.githubusercontent.com/usnistgov/jarvis_leaderboard/main/jarvis_leaderboard/benchmarks/alignn_model/metadata.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 391 [text/plain]
Saving to: ‘metadata.json’

metadata.json       100%[===================>]     391  --.-KB/s    in 0s      

2023-04-26 04:59:59 (8.15 MB/s) - ‘metadata.json’ saved [391/391]



In [24]:
from jarvis.db.jsonutils import loadjson,dumpjson
meta = loadjson('metadata.json')

In [25]:
meta

{'model_name': 'ALIGNN',
 'project_url': 'https://www.nature.com/articles/s41524-021-00650-1',
 'date_submitted': '01-14-2023',
 'author_email': 'knc6@nist.gov',
 'database_version': '12-12-2022',
 'team_name': 'JARVIS',
 'time_taken_seconds': 458.17,
 'language': 'python',
 'os': 'linux',
 'software_used': 'jarvis-tools,numpy,scipy,torch,alignn',
 'harware_used': 'nisaba-cluster at NIST, V100 Tesla GPU'}

In [27]:
meta['model_name'] = 'kgcnn_schnet2'
meta['project_url'] = 'https://github.com/aimat-lab/gcnn_keras/'
# TODO: Add other details
dumpjson(data=meta,filename='metadata.json')

In [28]:
!wget https://raw.githubusercontent.com/usnistgov/jarvis_leaderboard/main/jarvis_leaderboard/benchmarks/alignn_model/run.sh

--2023-04-26 05:00:12--  https://raw.githubusercontent.com/usnistgov/jarvis_leaderboard/main/jarvis_leaderboard/benchmarks/alignn_model/run.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 54 [text/plain]
Saving to: ‘run.sh’

run.sh              100%[===================>]      54  --.-KB/s    in 0s      

2023-04-26 05:00:12 (4.21 MB/s) - ‘run.sh’ saved [54/54]



In [29]:
os.chdir(cwd)

1. Replace knc6 by your own GitHub username.
2. You might have to add your [GitHub password/token](https://github.com/settings/tokens) twice at the command prompt after you run the following command.
3. This will create a new pull request at the parent repo: https://github.com/usnistgov/jarvis_leaderboard/pulls
4. After an admin approves the contribution, the benchmark will be added to the main website.


In [30]:
!jarvis_upload.py --your_benchmark_directory  kgcnn_schnet2 --github_username knc6

For help: jarvis_upload.py -h
Using GitHub username knc6
Forked_url https://github.com/knc6/jarvis_leaderboard
response <Response [404]>
Forking repo curl -u knc6 https://api.github.com/repos/usnistgov/jarvis_leaderboard/forks -d ''
Enter host password for user 'knc6':
{
  "id": 632766806,
  "node_id": "R_kgDOJbdBVg",
  "name": "jarvis_leaderboard",
  "full_name": "knc6/jarvis_leaderboard",
  "private": false,
  "owner": {
    "login": "knc6",
    "id": 16902896,
    "node_id": "MDQ6VXNlcjE2OTAyODk2",
    "avatar_url": "https://avatars.githubusercontent.com/u/16902896?v=4",
    "gravatar_id": "",
    "url": "https://api.github.com/users/knc6",
    "html_url": "https://github.com/knc6",
    "followers_url": "https://api.github.com/users/knc6/followers",
    "following_url": "https://api.github.com/users/knc6/following{/other_user}",
    "gists_url": "https://api.github.com/users/knc6/gists{/gist_id}",
    "starred_url": "https://api.github.com/users/knc6/starred{/owner}{/repo}",
    "su

The above script automatically:

   a) forks the jarvis_leaderboard repo from `https://github.com/usnistgov/jarvis_leaderboard` to your GitHub account,

   b) clones the forked jarvis_leaderboard from GitHub,  

   c) copies the `test_bench` folder in the `jarvis_leaderboard/benchmarks` folder,

   d) check if adding your benchmarks breaks the workflow using `rebuild.py` script,

   e) `git add`, `git commit`, and `git push` to the forked repo,

   f) create a pull request to the main usnistgov repo,

   g) an admin will approve the benchmark and new pages will be created.


In [31]:
pip freeze

absl-py==1.4.0
alabaster==0.7.13
albumentations==1.2.1
altair==4.2.2
anyio==3.6.2
appdirs==1.4.4
argon2-cffi==21.3.0
argon2-cffi-bindings==21.2.0
arviz==0.15.1
ase==3.22.1
astropy==5.2.2
astunparse==1.6.3
attrs==23.1.0
audioread==3.0.0
autograd==1.5
Babel==2.12.1
backcall==0.2.0
beautifulsoup4==4.11.2
bleach==6.0.0
blis==0.7.9
blosc2==2.0.0
bokeh==2.4.3
branca==0.6.0
Brotli==1.0.9
CacheControl==0.12.11
cached-property==1.5.2
cachetools==5.3.0
catalogue==2.0.8
certifi==2022.12.7
cffi==1.15.1
chardet==4.0.0
charset-normalizer==2.0.12
chex==0.1.7
click==8.1.3
cloudpickle==2.2.1
cmake==3.25.2
cmdstanpy==1.1.0
colorama==0.4.6
colorcet==3.0.1
colorlover==0.3.0
community==1.0.0b1
confection==0.0.4
cons==0.4.5
contextlib2==0.6.0.post1
contourpy==1.0.7
convertdate==2.4.0
cryptography==40.0.2
cufflinks==0.17.3
cvxopt==1.3.0
cvxpy==1.3.1
cycler==0.11.0
cymem==2.0.7
Cython==0.29.34
dask==2022.12.1
datascience==0.17.6
db-dtypes==1.1.1
dbus-python==1.2.16
debugpy==1.6.6
decorator==4.4.2
defusedxml==